### Importing modules and defining functions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import operator
import numpy as np
import seaborn as sns
bprp_bounds=[0.453,0.36,0.82,0.97,1.59,1.86,2.386,1.5,1.226,1.14,0.453]
gmag_bounds=[21,20.182,19.936,19.172,17.059,16.381,16.867,17.936,19.218,21,21]
gaiateamra=100.4029166667 #RA from Gaia DR2 Paper
gaiateamdec=-50.96611 #Dec from Gaia DR2 Paper

In [2]:
def weightedMean(x,xerr): #Takes lists of a value and a values uncertainty, returns 2 variables: the weighted mean, and the mean's uncertainty
    topfrac=0
    botfrac=0
    for i in range(len(x)):
        topfrac+=x[i]/(xerr[i]**2)
        botfrac+=1/(xerr[i]**2)
    mean=topfrac/botfrac
    mean_unc=np.sqrt(1/botfrac)
    return mean,mean_unc

In [3]:
def chiSquare(x,xerr,xmean): #Takes 2 lists and one value, the first list of values, the second list of the values uncertainties, and the third value is the mean value of the data set. Returns the chi squared value of that data set
    xchi=0
    for i in range(len(x)):
        xchi+=((x[i]-xmean)/xerr[i])**2
    return xchi

## DATA CRUNCHING 

In [4]:
def startup():
    print("Starting . . .")
    
    ###
    carinasurvey="OnePointSixDegreeRadius.csv" #The gaia data file with the survey of carina

    #quartertimesrhalflight.csv
    #half_times_rhalflight.csv
    #1timesrhalflight.csv
    #2timesrhalflight.csv
    #3timesrhalflight.csv
    #1degreeradius.csv
    #80percentdegree.csv

    #SELECT TOP 50000000 ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error FROM gaiadr2.gaia_source  WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',100.40291666666667,-50.96611111111111,1))=1    AND  (phot_g_mean_mag>=15 AND phot_rp_mean_mag>=15 AND phot_bp_mean_mag>=15 AND parallax<=2*parallax_error)
    
    ###
    gaiateamra=100.4029166667 #RA from Gaia DR2 Paper 
    gaiateamdec=-50.96611 #Dec from Gaia DR2 Paper 
    ###
    
    ###
    dfgaia=pd.read_csv(carinasurvey) #The Gaia data

    gaiara=dfgaia["ra"].values.tolist() #gaia ra list
    gaiara_error=dfgaia["ra_error"].values.tolist() #gaia ra error list
    gaiadec=dfgaia["dec"].values.tolist() #gaia dec list
    gaiadec_error=dfgaia["dec_error"].values.tolist() #gaia dec error list
    gaiapmra=dfgaia["pmra"].values.tolist() #gaia pmra list
    gaiapmra_error=dfgaia["pmra_error"].values.tolist() #gaia pmra error list
    gaiapmdec=dfgaia["pmdec"].values.tolist() #gaia pmdec list
    gaiapmdec_error=dfgaia["pmdec_error"].values.tolist() #gaia pmdec error list
    gaia_gmag=dfgaia["phot_g_mean_mag"].values.tolist() #gaia g mean mag list
    gaia_bmag=dfgaia["phot_bp_mean_mag"].values.tolist() #gaia b mean mag list
    gaia_rmag=dfgaia["phot_rp_mean_mag"].values.tolist() #gaia r mean mag list
    gaia_bminr=dfgaia["bp_rp"].values.tolist() #gaia b-r mag values
    gaia_bming=dfgaia["bp_g"].values.tolist() #gaia b-g mag values
    gaia_rv=dfgaia["radial_velocity"].values.tolist() #gaia radial velocity values
    gaia_rv_err=dfgaia["radial_velocity_error"].values.tolist() #gaia radial velocity error values
    gaia_gminr=dfgaia["g_rp"].values.tolist() #gaia g-r mag values
    
    ###
    
    # making the spherical coordinates correction
    for i in range(len(gaiara)):
        
        coss=np.sin(gaiateamdec)*np.sin(gaiadec[i])+np.cos(gaiateamdec)*np.cos(gaiadec[i])*np.cos(gaiara[i]-gaiateamra)
        
        newra=(np.cos(gaiadec[i])*np.sin(gaiara[i]-gaiateamra))/coss
        newdec=(np.cos(gaiateamdec)*np.sin(gaiadec[i])-np.sin(gaiateamdec)*np.cos(gaiadec[i])*np.cos(gaiara[i]-gaiateamra))/coss
        
        gaiara[i]=newra+gaiateamra
        gaiadec[i]=newdec+gaiateamdec
        
    ###
    
    #appending to lists only pmra and pmdec that are within 2sigma of the measured pmra and pmdec values

    teampmra=0.495 #gaia team values for carina
    teampmra_error=0.015
    teampmdec=0.143
    teampmdec_error=0.014

    pmra_error_quad=[] #pmra error lists with the gaia error added in quadrature
    pmdec_error_quad=[] #pmdec error lists with the gaia error added in quadrature

    for i in range(len(gaiapmra)): #Appending the gaia team pmra and pmdec to lists
        pmra_error_quad.append(np.sqrt((gaiapmra_error[i])**2+(teampmra_error)**2))
        pmdec_error_quad.append(np.sqrt(gaiapmdec_error[i]**2+teampmdec_error**2))

    #2 sigma limitation

    sigpmra=[]
    sigpmdec=[]
    sigpmra_error=[]
    sigpmdec_error=[]
    sig_gminr=[]
    sig_gmag=[]
    sig_bminr=[]

    signum=0.5

    for i in range(len(pmra_error_quad)):
        if gaiapmra[i]+signum*pmra_error_quad[i]>=teampmra and gaiapmra[i]-signum*pmra_error_quad[i]<=teampmra: #pmra check
            if gaiapmdec[i]+signum*pmdec_error_quad[i]>=teampmdec and gaiapmdec[i]-signum*pmdec_error_quad[i]<=teampmdec: #pmdec check
                sigpmra.append(gaiapmra[i])
                sigpmdec.append(gaiapmdec[i])
                sigpmra_error.append(gaiapmra_error[i])
                sigpmdec_error.append(gaiapmdec_error[i])
                sig_gminr.append(gaia_gminr[i])
                sig_gmag.append(gaia_gmag[i])
                sig_bminr.append(gaia_bminr[i])
    ###
    # y is gmag, x is bp-rp
    #01 eq'n: y=8.7957x+17.0155
    #12 eq'n: y=20.3745-0.534783x
    #23 eq'n: y=24.1125-5.09333x
    #34 eq'n: y=22.4778-3.40806x
    #45 eq'n: y=21.0517-2.51111x
    #56 eq'n: y=0.923954x+14.6624
    #67 eq'n: y=19.7458-1.20655x
    #78 eq'n: y=24.9542-4.67883x
    #89 eq'n: y=44.6219-20.7209x
    #90 eq'n: y=21

    branch_bp_rp=[]
    branch_gmag=[]
    nonbranch_bp_rp=[]
    nonbranch_gmag=[]

    count=0

    for i in range(len(gaia_bminr)):
        if gaia_bminr[i]>=bprp_bounds[1] and gaia_bminr[i]<bprp_bounds[0] and gaia_gmag[i]<=8.7957*gaia_bminr[i]+17.0155 and gaia_gmag[i]>=20.3745-0.534783*gaia_bminr[i]: #Polygon 1
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[0] and gaia_bminr[i]<bprp_bounds[2] and gaia_gmag[i]<=21 and gaia_gmag[i]>=20.3745-0.534783*gaia_bminr[i]: #Polygon 2
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[2] and gaia_bminr[i]<bprp_bounds[3] and gaia_gmag[i]<=21 and gaia_gmag[i]>=24.1125-5.09333*gaia_bminr[i]: #Polygon 3
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[3] and gaia_bminr[i]<bprp_bounds[9] and gaia_gmag[i]<=21 and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 4
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[9] and gaia_bminr[i]<bprp_bounds[8] and gaia_gmag[i]<=44.6219-20.7209*gaia_bminr[i] and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 5
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[8] and gaia_bminr[i]<bprp_bounds[7] and gaia_gmag[i]<=24.9542-4.67883*gaia_bminr[i] and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 6
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[7] and gaia_bminr[i]<bprp_bounds[4] and gaia_gmag[i]<=19.7458-1.20655*gaia_bminr[i] and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 7
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[4] and gaia_bminr[i]<bprp_bounds[5] and gaia_gmag[i]<=19.7458-1.20655*gaia_bminr[i] and gaia_gmag[i]>=21.0517-2.51111*gaia_bminr[i]: #Polygon 8
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[5] and gaia_bminr[i]<bprp_bounds[6] and gaia_gmag[i]<=19.7458-1.20655*gaia_bminr[i] and gaia_gmag[i]>=0.923954*gaia_bminr[i]+14.6624: #Polygon 9
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        else:
            nonbranch_bp_rp.append(gaia_bminr[i])
            nonbranch_gmag.append(gaia_gmag[i])
    ###
    branchmember_ra=[]
    branchmember_dec=[]
    branchmember_ra_error=[]
    branchmember_dec_error=[]
    branchmember_pmra=[]
    branchmember_pmra_error=[]
    branchmember_pmdec=[]
    branchmember_pmdec_error=[]
    branchmember_gmag=[]
    branchmember_bmag=[]
    branchmember_rmag=[]
    branchmember_bminr=[]
    branchmember_bming=[]
    branchmember_gminr=[]

    for i in range(len(gaia_bminr)): #Appending branch stars to lists with data on them 
        for j in range(len(branch_bp_rp)):
            if gaia_bminr[i]==branch_bp_rp[j] and gaia_gmag[i]==branch_gmag[j]:
                branchmember_ra.append(gaiara[i])
                branchmember_dec.append(gaiadec[i])
                branchmember_ra_error.append(gaiara_error[i])
                branchmember_dec_error.append(gaiadec_error[i])
                branchmember_pmra.append(gaiapmra[i])
                branchmember_pmra_error.append(gaiapmra_error[i])
                branchmember_pmdec.append(gaiapmdec[i])
                branchmember_pmdec_error.append(gaiapmdec_error[i])
                branchmember_gmag.append(gaia_gmag[i])
                branchmember_bmag.append(gaia_bmag[i])
                branchmember_rmag.append(gaia_rmag[i])
                branchmember_bminr.append(gaia_bminr[i])
                branchmember_bming.append(gaia_bming[i])
                branchmember_gminr.append(gaia_gminr[i])
    ###
    #Appending errors in quadrature
    branchmember_pmra_error_quad=[]
    branchmember_pmdec_error_quad=[]

    for i in range(len(branchmember_pmra_error)):
        branchmember_pmra_error_quad.append(np.sqrt((branchmember_pmra_error[i])**2+teampmra_error**2))
        branchmember_pmdec_error_quad.append(np.sqrt((branchmember_pmdec_error[i])**2+teampmdec_error**2))
    ###
    maglimit=eval(input("\nWhat magnitude limit do you want to impose? (Enter a number) "))
    ###
    #Lists to be filled with data from stars that are within the drawn branch, and are within the sigma cut

    sigbranchmember_ra=[]
    sigbranchmember_dec=[]
    sigbranchmember_ra_error=[]
    sigbranchmember_dec_error=[]
    sigbranchmember_pmra=[]
    sigbranchmember_pmra_error=[]
    sigbranchmember_pmdec=[]
    sigbranchmember_pmdec_error=[]
    sigbranchmember_gmag=[]
    sigbranchmember_bmag=[]
    sigbranchmember_rmag=[]
    sigbranchmember_bminr=[]
    sigbranchmember_bming=[]
    sigbranchmember_gminr=[]

    branchsignum=3

    beforecount=len(branchmember_ra)

    count=0

    # Added a gmagnitude cut of gmag<=19.1, as described in gaia dr2 collab paper

    for i in range(len(branchmember_pmra)):
        if branchmember_pmra[i]+branchsignum*branchmember_pmra_error_quad[i]>=teampmra and branchmember_pmra[i]-branchsignum*branchmember_pmra_error_quad[i]<=teampmra and branchmember_pmdec[i]+branchsignum*branchmember_pmdec_error_quad[i]>=teampmdec and branchmember_pmdec[i]-branchsignum*branchmember_pmdec_error_quad[i]<=teampmdec and branchmember_gmag[i]<=maglimit: #Checking pmra, then pmdec and brightness mag limits
            sigbranchmember_ra.append(branchmember_ra[i])
            sigbranchmember_dec.append(branchmember_dec[i])
            sigbranchmember_ra_error.append(branchmember_ra_error[i])
            sigbranchmember_dec_error.append(branchmember_dec_error[i])
            sigbranchmember_pmra.append(branchmember_pmra[i])
            sigbranchmember_pmra_error.append(branchmember_pmra_error_quad[i])
            sigbranchmember_pmdec.append(branchmember_pmdec[i])
            sigbranchmember_pmdec_error.append(branchmember_pmdec_error_quad[i])
            sigbranchmember_gmag.append(branchmember_gmag[i])
            sigbranchmember_bmag.append(branchmember_bmag[i])
            sigbranchmember_rmag.append(branchmember_rmag[i])
            sigbranchmember_bminr.append(branchmember_bminr[i])
            sigbranchmember_bming.append(branchmember_bming[i])
            sigbranchmember_gminr.append(branchmember_gminr[i])
            count+=1
    ###
    # Calculating weighted mean averages

    sigbranch_pmramean,sigbranch_pmramean_error=weightedMean(sigbranchmember_pmra,sigbranchmember_pmra_error)
    sigbranch_pmdecmean,sigbranch_pmdecmean_error=weightedMean(sigbranchmember_pmdec,sigbranchmember_pmdec_error)

    print("\nPMRA Weighted mean:",sigbranch_pmramean)
    print("PMRA Weighted mean error:",sigbranch_pmramean_error,"\n")
    print("PM-Dec Weighted mean:",sigbranch_pmdecmean)
    print("PM-Dec Weighted mean error:",sigbranch_pmdecmean_error,"\n")
    ###
    pmrachi=chiSquare(sigbranchmember_pmra,sigbranchmember_pmra_error,sigbranch_pmramean)
    pmdecchi=chiSquare(sigbranchmember_pmdec,sigbranchmember_pmdec_error,sigbranch_pmdecmean)

    print("n-1 d.o.f.:",len(sigbranchmember_pmra)-1)

    print("Chi Squared value for PM-RA:",pmrachi)
    print("Chi Squared value for PM-Dec:",pmdecchi,"\n")
    ###
    binnum=50

    ellipticity=0.33 #The ellipticity Taken from McConnachie paper

    rlist=[]

    ellipsearealist=[]

    semimajoraxis_size_list=[]
    semiminoraxis_size_list=[]

    rotationval=eval(input("What angle of rotation do you want to impose on the rotated coordinates? "))
    
    angle=np.deg2rad(rotationval)

    rotated_ra=[]
    rotated_dec=[]
    
    gaiateamra=100.4029166667 #RA from Gaia DR2 Paper 
    gaiateamdec=-50.96611 #Dec from Gaia DR2 Paper 

    for i in range(len(sigbranchmember_ra)):

        ra_rotated=(sigbranchmember_ra[i]-gaiateamra)*np.cos(angle)-(sigbranchmember_dec[i]-gaiateamdec)*np.sin(angle)
        dec_rotated=(sigbranchmember_dec[i]-gaiateamdec)*np.cos(angle)+(sigbranchmember_ra[i]-gaiateamra)*np.sin(angle)



        a=np.sqrt((ra_rotated)**2+(dec_rotated)**2/(1-ellipticity**2)) # Using this equation, calculating a single semimajor axis
        semimajoraxis_size_list.append(a) #Appending that semimajor axis value to a list
        b=np.sqrt((1-ellipticity**2)*(ra_rotated)**2+(dec_rotated)**2) # Using this equation, calculating a single semiminor axis
        semiminoraxis_size_list.append(b)
        r=np.sqrt((ra_rotated)**2+(dec_rotated)**2) # Radius calculation
        rlist.append(r)
        area=(semimajoraxis_size_list[i]*semiminoraxis_size_list[i]*(np.pi)) # Area given the semimajor and semiminor axis
        ellipsearealist.append(area)


        rotated_ra.append(ra_rotated+gaiateamra)
        rotated_dec.append(dec_rotated+gaiateamdec)

    #hope this works lol
    countodd=0
    counteven=0

    for i in range(len(rotated_ra)):
        if rotated_ra[i]<=gaiateamra and rotated_dec[i]<=gaiateamdec:
            counteven+=1
        elif rotated_ra[i]>=gaiateamra and rotated_dec[i]>=gaiateamdec:
            counteven+=1
        else:
            countodd+=1

    nratio=countodd/counteven

    print("\nThe ratio of quadrants:",nratio,"\n")
        
        
        
        #In this instance, i'm setting the mean ra and mean dec as (0,0)
        #This makes things easier, and all we're digging for here is semimajor axes and distance from te mean
    ###
    # Creating a dictionary to sort by the area's

    axisdict={}

    for i in range(len(ellipsearealist)):
        axisdict[i]=semimajoraxis_size_list[i]

    axis_sorted=sorted(axisdict.values())

    index_sorted=[]

    for i in range(len(semimajoraxis_size_list)):
        for j in range(len(axis_sorted)):
            if axis_sorted[i]==axisdict[j]:
                index_sorted.append(j) # Sorting the index values for using in the next for loop to append sorted semimajor axis values

    area_sorted = []

    for j in range(len(index_sorted)):
        area_sorted.append(ellipsearealist[index_sorted[j]])
    ###

    starsperbinnum=4
    
    raddensbinnum=int(len(ellipsearealist)/starsperbinnum)
    
    densitylist=[]
    semimajaxislist=[]
    unclist=[]

    old_area=0

    for i in range(raddensbinnum):
        
        semimajaxislist.append(axis_sorted[i*starsperbinnum+3])

        area=area_sorted[(i+1)*starsperbinnum-1]-old_area

        densitylist.append(starsperbinnum/area)

        old_area=area_sorted[(i+1)*starsperbinnum-1]
    
        unclist.append(np.sqrt(starsperbinnum)/area)
    
    
    ###
    print("Finished Startup! ")
    ###
    
    
    ### DECISION TREE STARTS HERE
    
    
    decision=input("What type of cut on the graphs do you want? Valid inputs are 'before', 'sigma cut', 'sigma and branch cut', 'all graphs', 'stat plots', or 'close'. To change the magnitude limit, enter 'maglim' ")
    
    while decision!="close":
        
        while decision=="maglim":
            ###
            maglimit=eval(input("\nWhat magnitude limit do you want to impose? (Enter a number) "))
            ###
            #Lists to be filled with data from stars that are within the drawn branch, and are within the sigma cut

            sigbranchmember_ra=[]
            sigbranchmember_dec=[]
            sigbranchmember_ra_error=[]
            sigbranchmember_dec_error=[]
            sigbranchmember_pmra=[]
            sigbranchmember_pmra_error=[]
            sigbranchmember_pmdec=[]
            sigbranchmember_pmdec_error=[]
            sigbranchmember_gmag=[]
            sigbranchmember_bmag=[]
            sigbranchmember_rmag=[]
            sigbranchmember_bminr=[]
            sigbranchmember_bming=[]
            sigbranchmember_gminr=[]

            branchsignum=3

            beforecount=len(branchmember_ra)

            count=0

            # Added a gmagnitude cut of gmag<=19.1, as described in gaia dr2 collab paper

            for i in range(len(branchmember_pmra)):
                if branchmember_pmra[i]+branchsignum*branchmember_pmra_error_quad[i]>=teampmra and branchmember_pmra[i]-branchsignum*branchmember_pmra_error_quad[i]<=teampmra and branchmember_pmdec[i]+branchsignum*branchmember_pmdec_error_quad[i]>=teampmdec and branchmember_pmdec[i]-branchsignum*branchmember_pmdec_error_quad[i]<=teampmdec and branchmember_gmag[i]<=maglimit: #Checking pmra, then pmdec and brightness mag limits
                    sigbranchmember_ra.append(branchmember_ra[i])
                    sigbranchmember_dec.append(branchmember_dec[i])
                    sigbranchmember_ra_error.append(branchmember_ra_error[i])
                    sigbranchmember_dec_error.append(branchmember_dec_error[i])
                    sigbranchmember_pmra.append(branchmember_pmra[i])
                    sigbranchmember_pmra_error.append(branchmember_pmra_error_quad[i])
                    sigbranchmember_pmdec.append(branchmember_pmdec[i])
                    sigbranchmember_pmdec_error.append(branchmember_pmdec_error_quad[i])
                    sigbranchmember_gmag.append(branchmember_gmag[i])
                    sigbranchmember_bmag.append(branchmember_bmag[i])
                    sigbranchmember_rmag.append(branchmember_rmag[i])
                    sigbranchmember_bminr.append(branchmember_bminr[i])
                    sigbranchmember_bming.append(branchmember_bming[i])
                    sigbranchmember_gminr.append(branchmember_gminr[i])
                    count+=1
            ###
            # Calculating weighted mean averages

            sigbranch_pmramean,sigbranch_pmramean_error=weightedMean(sigbranchmember_pmra,sigbranchmember_pmra_error)
            sigbranch_pmdecmean,sigbranch_pmdecmean_error=weightedMean(sigbranchmember_pmdec,sigbranchmember_pmdec_error)

            print("\nPMRA Weighted mean:",sigbranch_pmramean)
            print("PMRA Weighted mean error:",sigbranch_pmramean_error,"\n")
            print("PM-Dec Weighted mean:",sigbranch_pmdecmean)
            print("PM-Dec Weighted mean error:",sigbranch_pmdecmean_error,"\n")
            ###
            pmrachi=chiSquare(sigbranchmember_pmra,sigbranchmember_pmra_error,sigbranch_pmramean)
            pmdecchi=chiSquare(sigbranchmember_pmdec,sigbranchmember_pmdec_error,sigbranch_pmdecmean)

            print("n-1 d.o.f.:",len(sigbranchmember_pmra)-1)

            print("Chi Squared value for PM-RA:",pmrachi)
            print("Chi Squared value for PM-Dec:",pmdecchi,"\n")
            ###
            binnum=50

            ellipticity=0.33 #The ellipticity Taken from McConnachie paper

            gaiateamra=100.4029166667 #RA from Gaia DR2 Paper 
            gaiateamdec=-50.96611 #Dec from Gaia DR2 Paper 

            rlist=[]

            ellipsearealist=[]

            semimajoraxis_size_list=[]
            semiminoraxis_size_list=[]

            rotationval=eval(input("What angle of rotation do you want to impose on the rotated coordinates? "))
            
            angle=np.deg2rad(rotationval)

            rotated_ra=[]
            rotated_dec=[]

            for i in range(len(sigbranchmember_ra)):

                ra_rotated=(sigbranchmember_ra[i]-gaiateamra)*np.cos(angle)-(sigbranchmember_dec[i]-gaiateamdec)*np.sin(angle)
                dec_rotated=(sigbranchmember_dec[i]-gaiateamdec)*np.cos(angle)+(sigbranchmember_ra[i]-gaiateamra)*np.sin(angle)



                a=np.sqrt((ra_rotated)**2+(dec_rotated)**2/(1-ellipticity**2)) # Using this equation, calculating a single semimajor axis
                semimajoraxis_size_list.append(a) #Appending that semimajor axis value to a list
                b=np.sqrt((1-ellipticity**2)*(ra_rotated)**2+(dec_rotated)**2) # Using this equation, calculating a single semiminor axis
                semiminoraxis_size_list.append(b)
                r=np.sqrt((ra_rotated)**2+(dec_rotated)**2) # Radius calculation
                rlist.append(r)
                area=(semimajoraxis_size_list[i]*semiminoraxis_size_list[i]*(np.pi)) # Area given the semimajor and semiminor axis
                ellipsearealist.append(area)
                
                
                rotated_ra.append(ra_rotated+gaiateamra)
                rotated_dec.append(dec_rotated+gaiateamdec)


                #In this instance, i'm setting the mean ra and mean dec as (0,0)
                #This makes things easier, and all we're digging for here is semimajor axes and distance from te mean
            
            #hope this works lol
            
            countodd=0
            counteven=0

            for i in range(len(rotated_ra)):
                if rotated_ra[i]<=gaiateamra and rotated_dec[i]<=gaiateamdec:
                    counteven+=1
                elif rotated_ra[i]>=gaiateamra and rotated_dec[i]>=gaiateamdec:
                    counteven+=1
                else:
                    countodd+=1

            nratio=countodd/counteven

            print("\nThe ratio of quadrants:",nratio,"\n")
            
            ###
            
            
            # Creating a dictionary to sort by the area's

            axisdict={}

            for i in range(len(ellipsearealist)):
                axisdict[i]=semimajoraxis_size_list[i]

            axis_sorted=sorted(axisdict.values())

            index_sorted=[]

            for i in range(len(semimajoraxis_size_list)):
                for j in range(len(axis_sorted)):
                    if axis_sorted[i]==axisdict[j]:
                        index_sorted.append(j) # Sorting the index values for using in the next for loop to append sorted semimajor axis values

            area_sorted = []

            for j in range(len(index_sorted)):
                area_sorted.append(ellipsearealist[index_sorted[j]])
            ###

            raddensbinnum=int(len(ellipsearealist)/starsperbinnum)
    
            densitylist=[]
            semimajaxislist=[]
            unclist=[]

            old_area=0

            for i in range(raddensbinnum):

                semimajaxislist.append(axis_sorted[i*starsperbinnum+3])

                area=area_sorted[(i+1)*starsperbinnum-1]-old_area

                densitylist.append(starsperbinnum/area)

                old_area=area_sorted[(i+1)*starsperbinnum-1]

                unclist.append(np.sqrt(starsperbinnum)/area)

            

            ###
            print("Finished Startup! ")
            ###
            ###
            
            decision=input("\nWhat type of cut on the graphs do you want? Valid inputs are 'before', 'sigma cut', 'sigma and branch cut', 'all graphs', 'stat plots', or 'close'. If you want to change the magnitude limit, enter 'maglim' ")
        
        #THE DECISIONS TO PRODUCE SOME GRAPHS
        
        if decision=="before":
            seconddecision=input("What graph do you want? Valid inputs are 'gminrvsgmag' and 'propermotion' ")
            while seconddecision!="gminrvsgmag" and seconddecision!="propermotion":
                seconddecision=input("Incorrect entry. What graph do you want? Valid inputs are 'gminrvsgmag' and 'propermotion' ")            
            if seconddecision=="gminrvsgmag":
                gminr_gmag_graph(gaia_gminr,gaia_gmag)
            elif seconddecision=="propermotion":
                propermotion_graph_all(gaiapmra,gaiapmdec)

        elif decision=="sigma cut":
            seconddecision=input("What graph do you want? Valid inputs are 'gminrvsgmag', 'propermotion', or 'bminrvsgmag' ")
            while seconddecision!="gminrvsgmag" and seconddecision!="propermotion" and seconddecision!="bminrvsgmag":
                seconddecision=input("Incorrect entry. What graph do you want? valid inputs are 'gminrvsgmag', 'propermotion', or 'bminrvsgmag' ")
            if seconddecision=="gminrvsgmag":
                gminrvsgmag_sigcut(sig_gminr,sig_gmag)
            elif seconddecision=="propermotion":
                propermotion_sigcut(sigpmra,sigpmdec)
            elif seconddecision=="bminrvsgmag":
                bminrvsgmag_sigcut_graph(sig_bminr,sig_gmag,bprp_bounds,gmag_bounds)

        elif decision=="sigma and branch cut":
            seconddecision=input("What graph do you want? Valid inputs are 'bprpvsgmag', 'propermotion', 'positionplot', or 'rotated positionplot' ")
            while seconddecision!="bprpvsgmag" and seconddecision!="propermotion" and seconddecision!="positionplot" and seconddecision!="rotated positionplot":
                seconddecision=input("Incorrect entry. What graph do you want? Valid inputs are 'bprpvsgmag', 'propermotion', 'positionplot', or 'rotated positionplot' ")
            if seconddecision=="bprpvsgmag":
                bprpvsgmag_branch(branch_bp_rp,branch_gmag,nonbranch_bp_rp,nonbranch_gmag,bprp_bounds,gmag_bounds)
            elif seconddecision=="propermotion":
                propermotion_withmean(sigbranchmember_pmra,sigbranchmember_pmdec,sigbranch_pmramean,sigbranch_pmdecmean,sigbranch_pmdecmean_error,sigbranch_pmramean_error)
            elif seconddecision=="positionplot":
                postcuts_positionplot(sigbranchmember_ra,sigbranchmember_dec)
            elif seconddecision=="rotated positionplot":
                rotated_positionplot(rotated_ra,rotated_dec)

        elif decision=="all graphs":
            allGraphs(gaia_gminr,gaia_gmag,gaiapmra,gaiapmdec,sig_gminr,sig_gmag,sigpmra,sigpmdec,sig_bminr,branch_bp_rp,branch_gmag,sigbranchmember_pmra,sigbranchmember_pmdec,sigbranch_pmramean,sigbranch_pmdecmean,sigbranch_pmdecmean_error,sigbranch_pmramean_error,sigbranchmember_ra,sigbranchmember_dec,rotated_ra,rotated_dec,semimajoraxis_size_list,binnum,ellipsearealist,semimajaxislist,densitylist,nonbranch_bp_rp,nonbranch_gmag,bprp_bounds,gmag_bounds,unclist)

        elif decision=="stat plots":
            seconddecision=input("What graph do you want? Valid inputs are 'numberdensity positionplot', 'numberdensity propermotionplot', 'semimajoraxis histogram', 'ellipsearea histogram', or 'numberdensity semimajoraxis' ")
            while seconddecision!="numberdensity positionplot" and seconddecision!='numberdensity propermotionplot' and seconddecision!="semimajoraxis histogram" and seconddecision!="ellipsearea histogram" and seconddecision!="numberdensity semimajoraxis":
                seconddecision=input("Incorrect entry. What graph do you want? Valid inputs are 'numberdensity positionplot', 'numberdensity propermotionplot', 'semimajoraxis histogram', 'ellipsearea histogram', or 'numberdensity semimajoraxis' ")
            if seconddecision=="numberdensity positionplot":
                numberdensity_position_plot(sigbranchmember_ra, sigbranchmember_dec)
            elif seconddecision=="numberdensity propermotionplot":
                numberdensity_propermotion_plot(sigbranchmember_pmra, sigbranchmember_pmdec)
            elif seconddecision=="semimajoraxis histogram":
                semimajoraxis_histogram(semimajoraxis_size_list,binnum)
            elif seconddecision=="ellipsearea histogram":
                ellipsearea_histogram(ellipsearealist,binnum)
            elif seconddecision=="numberdensity semimajoraxis":
                semimajaxis_densityplot(semimajaxislist,densitylist,unclist)

        elif decision!="close":
            print("\nInvalid entry. Please enter a new input. ")
        
        decision=input("What type of cut on the graphs do you want? Valid inputs are 'before', 'sigma cut', 'sigma and branch cut', 'all graphs', 'stat plots', or 'close'. To change the maglimit, enter 'maglim' ")
        
            
    
    
    
    '''
    
    whichfile()
    dataframes()
    sigmaCut()
    branchCut()
    append_all_branchmember()
    errorquad()
    MAGLIMIT  FOR LOOP
    finalCut(maglimit)
    motionMeans()
    chiSquared_propermotion()
    rotatedCoords()
    areaSort()
    densitysort()
    print("Finished Startup!")
    decisionmaker()
    
    '''

In [5]:
def whichfile(): # a program to take the filename and return it
    carinasurvey="3timesrhalflight.csv" #The gaia data file with the survey of carina

    #quartertimesrhalflight.csv
    #half_times_rhalflight.csv
    #1timesrhalflight.csv
    #2timesrhalflight.csv
    #3timesrhalflight.csv

    #SELECT TOP 2000000 ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error FROM gaiadr2.gaia_source  WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',100.40291666666667,-50.96611111111111,0.03916666666666667))=1    AND  (phot_bp_mean_mag>=15 AND phot_rp_mean_mag>=15 AND phot_g_mean_mag>=15 AND parallax<=2*parallax_error

In [6]:
def dataframes(carinasurvey):

    dfgaia=pd.read_csv(carinasurvey) #The Gaia data

    gaiara=dfgaia["ra"].values.tolist() #gaia ra list
    gaiara_error=dfgaia["ra_error"].values.tolist() #gaia ra error list
    gaiadec=dfgaia["dec"].values.tolist() #gaia dec list
    gaiadec_error=dfgaia["dec_error"].values.tolist() #gaia dec error list
    gaiapmra=dfgaia["pmra"].values.tolist() #gaia pmra list
    gaiapmra_error=dfgaia["pmra_error"].values.tolist() #gaia pmra error list
    gaiapmdec=dfgaia["pmdec"].values.tolist() #gaia pmdec list
    gaiapmdec_error=dfgaia["pmdec_error"].values.tolist() #gaia pmdec error list
    gaia_gmag=dfgaia["phot_g_mean_mag"].values.tolist() #gaia g mean mag list
    gaia_bmag=dfgaia["phot_bp_mean_mag"].values.tolist() #gaia b mean mag list
    gaia_rmag=dfgaia["phot_rp_mean_mag"].values.tolist() #gaia r mean mag list
    gaia_bminr=dfgaia["bp_rp"].values.tolist() #gaia b-r mag values
    gaia_bming=dfgaia["bp_g"].values.tolist() #gaia b-g mag values
    gaia_rv=dfgaia["radial_velocity"].values.tolist() #gaia radial velocity values
    gaia_rv_err=dfgaia["radial_velocity_error"].values.tolist() #gaia radial velocity error values
    gaia_gminr=dfgaia["g_rp"].values.tolist() #gaia g-r mag values

In [7]:
def sigmaCut():
    #appending to lists only pmra and pmdec that are within 2sigma of the measured pmra and pmdec values

    teampmra=0.495 #gaia team values for carina
    teampmra_error=0.015
    teampmdec=0.143
    teampmdec_error=0.014

    pmra_error_quad=[] #pmra error lists with the gaia error added in quadrature
    pmdec_error_quad=[] #pmdec error lists with the gaia error added in quadrature

    for i in range(len(gaiapmra)): #Appending the gaia team pmra and pmdec to lists
        pmra_error_quad.append(np.sqrt((gaiapmra_error[i])**2+(teampmra_error)**2))
        pmdec_error_quad.append(np.sqrt(gaiapmdec_error[i]**2+teampmdec_error**2))

    #2 sigma limitation

    sigpmra=[]
    sigpmdec=[]
    sigpmra_error=[]
    sigpmdec_error=[]
    sig_gminr=[]
    sig_gmag=[]
    sig_bminr=[]

    signum=0.5

    for i in range(len(pmra_error_quad)):
        if gaiapmra[i]+signum*pmra_error_quad[i]>=teampmra and gaiapmra[i]-signum*pmra_error_quad[i]<=teampmra: #pmra check
            if gaiapmdec[i]+signum*pmdec_error_quad[i]>=teampmdec and gaiapmdec[i]-signum*pmdec_error_quad[i]<=teampmdec: #pmdec check
                sigpmra.append(gaiapmra[i])
                sigpmdec.append(gaiapmdec[i])
                sigpmra_error.append(gaiapmra_error[i])
                sigpmdec_error.append(gaiapmdec_error[i])
                sig_gminr.append(gaia_gminr[i])
                sig_gmag.append(gaia_gmag[i])
                sig_bminr.append(gaia_bminr[i])

In [8]:
def branchCut():
    
    # y is gmag, x is bp-rp
    #01 eq'n: y=8.7957x+17.0155
    #12 eq'n: y=20.3745-0.534783x
    #23 eq'n: y=24.1125-5.09333x
    #34 eq'n: y=22.4778-3.40806x
    #45 eq'n: y=21.0517-2.51111x
    #56 eq'n: y=0.923954x+14.6624
    #67 eq'n: y=19.7458-1.20655x
    #78 eq'n: y=24.9542-4.67883x
    #89 eq'n: y=44.6219-20.7209x
    #90 eq'n: y=21

    branch_bp_rp=[]
    branch_gmag=[]
    nonbranch_bp_rp=[]
    nonbranch_gmag=[]

    count=0

    for i in range(len(gaia_bminr)):
        if gaia_bminr[i]>=bprp_bounds[1] and gaia_bminr[i]<bprp_bounds[0] and gaia_gmag[i]<=8.7957*gaia_bminr[i]+17.0155 and gaia_gmag[i]>=20.3745-0.534783*gaia_bminr[i]: #Polygon 1
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[0] and gaia_bminr[i]<bprp_bounds[2] and gaia_gmag[i]<=21 and gaia_gmag[i]>=20.3745-0.534783*gaia_bminr[i]: #Polygon 2
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[2] and gaia_bminr[i]<bprp_bounds[3] and gaia_gmag[i]<=21 and gaia_gmag[i]>=24.1125-5.09333*gaia_bminr[i]: #Polygon 3
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[3] and gaia_bminr[i]<bprp_bounds[9] and gaia_gmag[i]<=21 and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 4
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[9] and gaia_bminr[i]<bprp_bounds[8] and gaia_gmag[i]<=44.6219-20.7209*gaia_bminr[i] and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 5
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[8] and gaia_bminr[i]<bprp_bounds[7] and gaia_gmag[i]<=24.9542-4.67883*gaia_bminr[i] and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 6
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[7] and gaia_bminr[i]<bprp_bounds[4] and gaia_gmag[i]<=19.7458-1.20655*gaia_bminr[i] and gaia_gmag[i]>=22.4778-3.40806*gaia_bminr[i]: #Polygon 7
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[4] and gaia_bminr[i]<bprp_bounds[5] and gaia_gmag[i]<=19.7458-1.20655*gaia_bminr[i] and gaia_gmag[i]>=21.0517-2.51111*gaia_bminr[i]: #Polygon 8
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        elif gaia_bminr[i]>=bprp_bounds[5] and gaia_bminr[i]<bprp_bounds[6] and gaia_gmag[i]<=19.7458-1.20655*gaia_bminr[i] and gaia_gmag[i]>=0.923954*gaia_bminr[i]+14.6624: #Polygon 9
            count+=1
            branch_bp_rp.append(gaia_bminr[i])
            branch_gmag.append(gaia_gmag[i])
        else:
            nonbranch_bp_rp.append(gaia_bminr[i])
            nonbranch_gmag.append(gaia_gmag[i])

In [9]:
def append_all_branchmember():
    branchmember_ra=[]
    branchmember_dec=[]
    branchmember_ra_error=[]
    branchmember_dec_error=[]
    branchmember_pmra=[]
    branchmember_pmra_error=[]
    branchmember_pmdec=[]
    branchmember_pmdec_error=[]
    branchmember_gmag=[]
    branchmember_bmag=[]
    branchmember_rmag=[]
    branchmember_bminr=[]
    branchmember_bming=[]
    branchmember_gminr=[]

    for i in range(len(gaia_bminr)): #Appending branch stars to lists with data on them 
        for j in range(len(branch_bp_rp)):
            if gaia_bminr[i]==branch_bp_rp[j] and gaia_gmag[i]==branch_gmag[j]:
                branchmember_ra.append(gaiara[i])
                branchmember_dec.append(gaiadec[i])
                branchmember_ra_error.append(gaiara_error[i])
                branchmember_dec_error.append(gaiadec_error[i])
                branchmember_pmra.append(gaiapmra[i])
                branchmember_pmra_error.append(gaiapmra_error[i])
                branchmember_pmdec.append(gaiapmdec[i])
                branchmember_pmdec_error.append(gaiapmdec_error[i])
                branchmember_gmag.append(gaia_gmag[i])
                branchmember_bmag.append(gaia_bmag[i])
                branchmember_rmag.append(gaia_rmag[i])
                branchmember_bminr.append(gaia_bminr[i])
                branchmember_bming.append(gaia_bming[i])
                branchmember_gminr.append(gaia_gminr[i])

In [10]:
def finalCut(maglimit):

    #Lists to be filled with data from stars that are within the drawn branch, and are within the sigma cut

    sigbranchmember_ra=[]
    sigbranchmember_dec=[]
    sigbranchmember_ra_error=[]
    sigbranchmember_dec_error=[]
    sigbranchmember_pmra=[]
    sigbranchmember_pmra_error=[]
    sigbranchmember_pmdec=[]
    sigbranchmember_pmdec_error=[]
    sigbranchmember_gmag=[]
    sigbranchmember_bmag=[]
    sigbranchmember_rmag=[]
    sigbranchmember_bminr=[]
    sigbranchmember_bming=[]
    sigbranchmember_gminr=[]

    branchsignum=3

    beforecount=len(branchmember_ra)

    count=0

    # Added a gmagnitude cut of gmag<=19.1, as described in gaia dr2 collab paper

    for i in range(len(branchmember_pmra)):
        if branchmember_pmra[i]+branchsignum*branchmember_pmra_error_quad[i]>=teampmra and branchmember_pmra[i]-branchsignum*branchmember_pmra_error_quad[i]<=teampmra and branchmember_pmdec[i]+branchsignum*branchmember_pmdec_error_quad[i]>=teampmdec and branchmember_pmdec[i]-branchsignum*branchmember_pmdec_error_quad[i]<=teampmdec and branchmember_gmag[i]<=maglimit: #Checking pmra, then pmdec and 
            sigbranchmember_ra.append(branchmember_ra[i])
            sigbranchmember_dec.append(branchmember_dec[i])
            sigbranchmember_ra_error.append(branchmember_ra_error[i])
            sigbranchmember_dec_error.append(branchmember_dec_error[i])
            sigbranchmember_pmra.append(branchmember_pmra[i])
            sigbranchmember_pmra_error.append(branchmember_pmra_error_quad[i])
            sigbranchmember_pmdec.append(branchmember_pmdec[i])
            sigbranchmember_pmdec_error.append(branchmember_pmdec_error_quad[i])
            sigbranchmember_gmag.append(branchmember_gmag[i])
            sigbranchmember_bmag.append(branchmember_bmag[i])
            sigbranchmember_rmag.append(branchmember_rmag[i])
            sigbranchmember_bminr.append(branchmember_bminr[i])
            sigbranchmember_bming.append(branchmember_bming[i])
            sigbranchmember_gminr.append(branchmember_gminr[i])
            count+=1

In [11]:
def motionMeans():
    # Calculating weighted mean averages

    sigbranch_pmramean,sigbranch_pmramean_error=weightedMean(sigbranchmember_pmra,sigbranchmember_pmra_error)
    sigbranch_pmdecmean,sigbranch_pmdecmean_error=weightedMean(sigbranchmember_pmdec,sigbranchmember_pmdec_error)

    print("PMRA Weighted mean:",sigbranch_pmramean)
    print("PMRA Weighted mean error:",sigbranch_pmramean_error,"\n")
    print("PM-Dec Weighted mean:",sigbranch_pmdecmean)
    print("PM-Dec Weighted mean error:",sigbranch_pmdecmean_error)

In [12]:
def errorquad():
    #Appending errors in quadrature
    branchmember_pmra_error_quad=[]
    branchmember_pmdec_error_quad=[]

    for i in range(len(branchmember_pmra_error)):
        branchmember_pmra_error_quad.append(np.sqrt((branchmember_pmra_error[i])**2+teampmra_error**2))
        branchmember_pmdec_error_quad.append(np.sqrt((branchmember_pmdec_error[i])**2+teampmdec_error**2))

In [13]:
def chiSquared_propermotion():
    pmrachi=chiSquare(sigbranchmember_pmra,sigbranchmember_pmra_error,sigbranch_pmramean)
    pmdecchi=chiSquare(sigbranchmember_pmdec,sigbranchmember_pmdec_error,sigbranch_pmdecmean)

    print("n-1 d.o.f.:",len(sigbranchmember_pmra)-1)

    print("Chi Squared value for PM-RA:",pmrachi)
    print("Chi Squared value for PM-Dec:",pmdecchi)

In [14]:
def rotatedCoords():
    binnum=50

    ellipticity=0.33 #The ellipticity Taken from McConnachie paper

    gaiateamra=100.4029166667 #RA from Gaia DR2 Paper 
    gaiateamdec=-50.96611 #Dec from Gaia DR2 Paper 

    rlist=[]

    ellipsearealist=[]

    semimajoraxis_size_list=[]
    semiminoraxis_size_list=[]

    rotationval=eval(input("What angle of rotation do you want to impose on the rotated coordinates? "))
    
    angle=np.deg2rad(rotationval)

    rotated_ra=[]
    rotated_dec=[]

    for i in range(len(sigbranchmember_ra)):

        ra_rotated=(sigbranchmember_ra[i]-gaiateamra)*np.cos(angle)-(sigbranchmember_dec[i]-gaiateamdec)*np.sin(angle)
        dec_rotated=(sigbranchmember_dec[i]-gaiateamdec)*np.cos(angle)+(sigbranchmember_ra[i]-gaiateamra)*np.sin(angle)

        rotated_ra.append(ra_rotated)
        rotated_dec.append(dec_rotated)

        a=np.sqrt((rotated_ra[i])**2+(rotated_dec[i])**2/(1-ellipticity**2)) # Using this equation, calculating a single semimajor axis
        semimajoraxis_size_list.append(a) #Appending that semimajor axis value to a list
        b=np.sqrt((1-ellipticity**2)*(rotated_ra[i])**2+(rotated_dec[i])**2) # Using this equation, calculating a single semiminor axis
        semiminoraxis_size_list.append(b)
        r=np.sqrt((rotated_ra[i])**2+(rotated_dec[i])**2) # Radius calculation
        rlist.append(r)
        area=(semimajoraxis_size_list[i]*semiminoraxis_size_list[i]*(np.pi)) # Area given the semimajor and semiminor axis
        ellipsearealist.append(area)
    
    countodd=0
    counteven=0

    for i in range(len(rotated_ra)):
        if rotated_ra[i]<=gaiateamra and rotated_dec[i]<=gaiateamdec:
            counteven+=1
        elif rotated_ra[i]>=gaiateamra and rotated_dec[i]>=gaiateamdec:
            counteven+=1
        else:
            countodd+=1

    nratio=countodd/counteven

    print("\nThe ratio of quadrants:",nratio,"\n")

        #In this instance, i'm setting the mean ra and mean dec as (0,0)
        #This makes things easier, and all we're digging for here is semimajor axes and distance from te mean

In [15]:
def areaSort():

    # Creating a dictionary to sort by the area's

    areadict={}

    for i in range(len(ellipsearealist)):
        areadict[i]=ellipsearealist[i]

    area_sorted=sorted(areadict.values())

    index_sorted=[]

    for i in range(len(ellipsearealist)):
        for j in range(len(area_sorted)):
            if area_sorted[i]==areadict[j]:
                index_sorted.append(j) # Sorting the index values for using in the next for loop to append sorted semimajor axis values

    semimajoraxis_sorted = []

    for j in range(len(index_sorted)):
        semimajoraxis_sorted.append(semimajoraxis_size_list[index_sorted[j]])

In [16]:
def densitysort():

    starsperbinnum=4
    raddensbinnum=int(len(ellipsearealist)/starsperbinnum)
    
    densitylist=[]
    semimajaxislist=[]
    unclist=[]

    old_area=0

    for i in range(raddensbinnum):
        
        semimajaxislist.append(axis_sorted[i*starsperbinnum+3])

        area=area_sorted[(i+1)*starsperbinnum-1]-old_area

        densitylist.append(starsperbinnum/area)

        old_area=area_sorted[(i+1)*starsperbinnum-1]
    
        unclist.append(np.sqrt(starsperbinnum)/area)

## GRAPHS

In [17]:
def allGraphs(gaia_gminr,gaia_gmag,gaiapmra,gaiapmdec,sig_gminr,sig_gmag,sigpmra,sigpmdec,sig_bminr,branch_bp_rp,branch_gmag,sigbranchmember_pmra,sigbranchmember_pmdec,sigbranch_pmramean,sigbranch_pmdecmean,sigbranch_pmdecmean_error,sigbranch_pmramean_error,sigbranchmember_ra,sigbranchmember_dec,rotated_ra,rotated_dec,semimajoraxis_size_list,binnum,ellipsearealist,semimajaxislist,densitylist,nonbranch_bp_rp,nonbranch_gmag,bprp_bounds,gmag_bounds,unclist):
    gminr_gmag_graph(gaia_gminr,gaia_gmag) #no cut
    propermotion_graph_all(gaiapmra,gaiapmdec) #no cut
    gminrvsgmag_sigcut(sig_gminr,sig_gmag) #sigma cut
    propermotion_sigcut(sigpmra,sigpmdec) #sigma cut 
    bminrvsgmag_sigcut_graph(sig_bminr,sig_gmag,bprp_bounds,gmag_bounds) #sigma cut
    bprpvsgmag_branch(branch_bp_rp,branch_gmag,nonbranch_bp_rp,nonbranch_gmag,bprp_bounds,gmag_bounds) #full cut 
    propermotion_withmean(sigbranchmember_pmra,sigbranchmember_pmdec,sigbranch_pmramean,sigbranch_pmdecmean,sigbranch_pmdecmean_error,sigbranch_pmramean_error) #full cut
    postcuts_positionplot(sigbranchmember_ra,sigbranchmember_dec) #full cut #
    numberdensity_position_plot(sigbranchmember_ra, sigbranchmember_dec) #stat plot
    numberdensity_propermotion_plot(sigbranchmember_pmra, sigbranchmember_pmdec) #stat plot
    rotated_positionplot(rotated_ra,rotated_dec) #full cut #
    semimajoraxis_histogram(semimajoraxis_size_list,binnum) #stat plot
    ellipsearea_histogram(ellipsearealist,binnum) #stat plot
    semimajaxis_densityplot(semimajaxislist,densitylist,unclist) #stat plot

In [18]:
def gminr_gmag_graph(gaia_gminr,gaia_gmag):
    
    plt.grid(True)

    plt.scatter(gaia_gminr,gaia_gmag,s=1,c="blue")

    plt.xlabel("g-r")
    plt.ylabel("g")

    plt.ylim(plt.ylim()[::-1])


    plt.title("g-r magnitude vs. g magnitude")

    #plt.gca().set_aspect('equal', adjustable='box')

    plt.savefig("bmingVSgmag.png",dpi=300)

    plt.show()

In [19]:
def propermotion_graph_all(gaiapmra,gaiapmdec):
    plt.scatter(gaiapmra,gaiapmdec,s=1,c="blue")

    plt.grid(True)

    plt.xlabel("PM-RA")
    plt.ylabel("PM-Dec")

    plt.xlim(-5,5)
    plt.ylim(-5,5)

    plt.xlim(plt.xlim()[::-1])

    plt.savefig("propermotiondiagram.png",dpi=300)

    plt.show()

In [20]:
def gminrvsgmag_sigcut(sig_gminr,sig_gmag):
    plt.scatter(sig_gminr,sig_gmag,s=1,c="blue")

    plt.grid(True)

    plt.xlabel("g-r")
    plt.ylabel("g")

    plt.ylim(plt.ylim()[::-1])

    plt.title("g-r magnitude vs. g magnitude")

    plt.savefig("bmingVSgmag_withsigcut.png",dpi=300)

    plt.show()

In [21]:
def propermotion_sigcut(sigpmra,sigpmdec):
    plt.scatter(sigpmra,sigpmdec,s=1,c="blue")

    plt.grid(True)

    plt.xlim(-5,5)
    plt.ylim(-5,5)

    plt.xlabel("μ$_α$ cos(δ) (mas yr$^{-1}$)")
    plt.ylabel("μ$_δ$ (mas yr$^{-1}$)")
    plt.title("Proper Motion of Carina")
    plt.gca().set_aspect('equal', adjustable='box')

    plt.xlim(plt.xlim()[::-1])

    plt.savefig("propermotiondiagram_withsigcut.png",dpi=300)

    plt.show()

In [22]:
def bminrvsgmag_sigcut_graph(sig_bminr,sig_gmag,bprp_bounds,gmag_bounds):
    plt.scatter(sig_bminr,sig_gmag,s=1,c="blue")
    plt.plot(bprp_bounds,gmag_bounds,"-o",linewidth=1,c="blue")

    plt.grid(True)

    plt.xlabel("bp-rp")
    plt.ylabel("g")

    plt.ylim(plt.ylim()[::-1])

    plt.title("bp-rp magnitude vs. g magnitude")

    plt.savefig("bpminrpVSgmag_withsigcut.png",dpi=300)

    plt.show()

In [23]:
def bprpvsgmag_branch(branch_bp_rp,branch_gmag,nonbranch_bp_rp,nonbranch_gmag,bprp_bounds,gmag_bounds):
    plt.scatter(branch_bp_rp,branch_gmag,s=1,c="blue")
    plt.scatter(nonbranch_bp_rp,nonbranch_gmag,s=1,c="red")
    plt.plot(bprp_bounds,gmag_bounds,c="blue")

    plt.grid(True)

    plt.xlabel("bp-rp")
    plt.ylabel("g")

    plt.ylim(plt.ylim(16.3,21.2)[::-1])
    plt.xlim(-0.3,2.55)

    plt.title("bp-rp magnitude vs. g magnitude")

    plt.savefig("bpminrpVSgmag_shapeselect.png",dpi=300)

    plt.show()

In [24]:
def propermotion_withmean(sigbranchmember_pmra,sigbranchmember_pmdec,sigbranch_pmramean,sigbranch_pmdecmean,sigbranch_pmdecmean_error,sigbranch_pmramean_error):
    plt.scatter(sigbranchmember_pmra,sigbranchmember_pmdec,s=1,c="blue")
#     plt.errorbar(sigbranchmember_pmra,sigbranchmember_pmdec,sigbranchmember_pmdec_error,sigbranchmember_pmra_error,linestyle="none",capsize=1,capthick=1,elinewidth=1,c="blue")

    plt.scatter(sigbranch_pmramean,sigbranch_pmdecmean,s=1,c="black")
    plt.errorbar(sigbranch_pmramean,sigbranch_pmdecmean,sigbranch_pmdecmean_error,sigbranch_pmramean_error,linestyle="none",capsize=1,capthick=1,elinewidth=1,c="black")

    plt.grid(True)

    plt.xlabel("μ$_α$ cos(δ) (mas yr$^{-1}$)")
    plt.ylabel("μ$_δ$ (mas yr$^{-1}$)")
    plt.title("Proper Motion of Carina")
    plt.gca().set_aspect('equal', adjustable='box')

    sigbranch_graphlim=2

    plt.xlim(-sigbranch_graphlim,sigbranch_graphlim)
    plt.ylim(-sigbranch_graphlim,sigbranch_graphlim)

    plt.xlim(plt.xlim()[::-1])

    plt.savefig("propermotiondiagram_withsigandbranchcut.png",dpi=300)

    plt.show()

In [25]:
def postcuts_positionplot(sigbranchmember_ra,sigbranchmember_dec):
    plt.scatter(sigbranchmember_ra,sigbranchmember_dec,s=1,c="blue")

    # plt.scatter(gaiateamra,gaiateamdec,c="black",s=1)

    plt.grid(True)

    plt.xlabel("α")
    plt.ylabel("δ")
    plt.title("Position of Carina")

    plt.xlim(plt.xlim()[::-1])

    plt.axis('tight')

    plt.savefig("position_siggcutmembcut.png",dpi=300)

    plt.show()

In [26]:
def numberdensity_position_plot(sigbranchmember_ra, sigbranchmember_dec):
    # Getting a number density plot of RA and Dec

    sns.set_style("white")
    kdeplot_raVSdec=sns.kdeplot(sigbranchmember_ra, sigbranchmember_dec,bw=0.035,kernel="gau")

    radecbin=kdeplot_raVSdec.get_figure()

    plt.xlabel("α")
    plt.ylabel("δ")
    plt.title("Position Diagram Number Density Plot")

    plt.grid(True)

    # sns.jointplot(sigbranchmember_ra,sigbranchmember_dec,kind="kde")

    plt.show()

    radecbin.savefig("kde_plot_RAvsDec.png",dpi=300)

In [27]:
def numberdensity_propermotion_plot(sigbranchmember_pmra, sigbranchmember_pmdec):
    # Getting a number density plot of PM-RA and PM-Dec

    sns.set_style("white")
    kdeplot_pmraVSpmdec=sns.kdeplot(sigbranchmember_pmra, sigbranchmember_pmdec,bw=0.065,kernel="gau")

    pmrapmdecbin=kdeplot_pmraVSpmdec.get_figure()

    plt.xlabel("μ$_α$ cos(δ) (mas yr$^{-1}$)")
    plt.ylabel("μ$_δ$ (mas yr$^{-1}$)")
    plt.title("Proper Motion of Carina Number Density Plot")
    # plt.gca().set_aspect('equal', adjustable='box')

    plt.xlim(plt.xlim()[::-1])

    plt.grid(True)

    plt.show()

    pmrapmdecbin.savefig("kde_plot_PMRAvsPMDec.png",dpi=300)

In [28]:
def rotated_positionplot(rotated_ra,rotated_dec):
    plt.scatter(rotated_ra,rotated_dec,s=1, c="blue")

    plt.grid(True)

    plt.xlabel("α")
    plt.ylabel("δ")
    plt.title("Position of Carina")
    
    plt.xlim(gaiateamra-1,gaiateamra+1)
    plt.ylim(gaiateamdec-1,gaiateamdec+1)

    plt.xlim(plt.xlim()[::-1])

    plt.savefig("rotatedpositionplot.png",dpi=300)
    
    plt.show()

In [29]:
def semimajoraxis_histogram(semimajoraxis_size_list,binnum):
    plt.hist(semimajoraxis_size_list,bins=binnum,color="blue")

    plt.xlabel("Semi-Major Axis length (degrees)")
    plt.ylabel("Density")
    plt.title("Number Density of the Semi-Major Axes of Carina Member stars")

    plt.savefig("histogram_of_axes_lengths.png",dpi=300)

    plt.show()

In [30]:
def ellipsearea_histogram(ellipsearealist,binnum):
    plt.hist(ellipsearealist,bins=binnum,color="blue")

    plt.xlabel("Ellipse Area")
    plt.ylabel("Density")
    plt.title("Number Density of the Area of the Ellipse of Carina Member Stars")

    plt.savefig("histogram_of_ellipse_areas.png",dpi=300)

    plt.show()

In [ ]:
def semimajaxis_densityplot(semimajaxislist,densitylist,unclist):
    
    plt.scatter(semimajaxislist,densitylist,s=1,c="blue")
    plt.errorbar(semimajaxislist,densitylist,unclist,0,elinewidth=1,capthick=1,c="blue",linestyle="none",capsize=1)

    plt.xlabel("Semi-Major Axis distance (degrees)")
    plt.ylabel("Radial number Density")
    
    plt.loglog()
    
    plt.savefig("Radial_surface_density_plot.png",dpi=300)

    plt.show()

## STARTING THE PROGRAM

In [ ]:
startup()

Starting . . .


In [ ]:
# Look at the position plots for the tangent plane approximation. Something is off in the math there.

# find a package that gives a maximum likelihood (?)